In [1]:
x = var('x')
x_l = var('x_l')
x_r = var('x_r')

t = var('t')
T = var('T')

s_l = var('s_l')
s_r = var('s_r')
s = var('s')

q = var('q')
q_l = var('q_l')
q_r = var('q_r')
q_s = var('q_s')

f = var('f')
# f_l = f(q_l), f_r = f(q_r), f_s = f(q_s)
f_l = var('f_l')
f_r = var('f_r')
f_s = var('f_s')

n_1 = vector([0, -1])
n_2 = vector([1, 0])
n_3 = vector([0, 1])
n_4 = vector([-1, 0])

u = vector([q, f(q)])

# s_1 = \dintt{0}{T}{u . n_1}{t} = \dintt{0}{T}{-f(q(x = x_l)}{t} = - f(q_l) T
s_1 = -1 * f_l * T
# s_2 = \dintt{x_l}{0}{u . n_2}{x} = \dintt{x_l}{s_l T}{q_l}{x} + \dintt{s_l T}{0}{q_s}{x} = q_l (s_l T - x_l) + q_s(- s_l T)
s_2 = q_l * (s_l * T - x_l) + q_s * (-1 * s_l * T)
# s_3 = \dintt{0}{x_r}{u . n_2}{x} = \dintt{0}{s_r T}{q_s}{x} + \dintt{s_r T}{x_r}{q_r}{x} = q_s (- s_r T) + q_r (x_r - s_r T)
s_3 = q_s * (s_r * T) + q_r * (x_r - s_r * T)
# s_4 = \dintt{0}{T}{u . n_3}{t} = \dintt{0}{T}{f(q_r)}{t} = f(q_r) T
s_4 = f_r * T
# s_5 = \dintt{0}{x_r}{u . n_4}{x} = \dintt{0}{x_r}{-q_r}{x} = -q_r x_r
s_5 = -1 * q_r * x_r
# s_6 = \dintt{x_l}{0}{u . n_4}{x} = \dintt{x_l}{0}{-q_l}{x} = q_l x_l
s_6 = q_l * x_l
# s_7 = \dintt{0}{T}{u . n_1}{t} = \dintt{0}{T}{-f(q_s)}{t} = - f(q_s) T
s_7 = -1 * f_s * T
# s_8 = \dintt{0}{T}{u . n_3}{t} = \dintt{0}{T}{f(q_s)}{t} = f(q_s) T
s_8 = f_s * T
left_cell = s_1 + s_2 + s_8 + s_6
right_cell = s_3 + s_4 + s_5 + s_7
both_cells = s_1 + s_2 + s_3 + s_4 + s_5 + s_6
print(left_cell.simplify_full())
print(right_cell.simplify_full())
print(both_cells.simplify_full())

-T*f_l + T*f_s + (T*q_l - T*q_s)*s_l
T*f_r - T*f_s - (T*q_r - T*q_s)*s_r
-T*f_l + T*f_r + (T*q_l - T*q_s)*s_l - (T*q_r - T*q_s)*s_r


## HLL Riemann Solver Conservative

In [2]:
rhs = left_cell + right_cell
sol = solve(rhs == 0, q_s)
print(sol[0])

q_s == (q_l*s_l - q_r*s_r - f_l + f_r)/(s_l - s_r)


In [9]:
q_s_dict = {q_s : (q_l*s_l - q_r*s_r - f_l + f_r)/(s_l - s_r)}

In [11]:
rhs = left_cell - right_cell
sol2 = solve(rhs == 0, f_s)
print(sol2[0].subs(q_s_dict).simplify_full())

f_s == (f_r*s_l + ((q_l - q_r)*s_l - f_l)*s_r)/(s_l - s_r)


In [8]:
type(sol[0])

<class 'sage.symbolic.expression.Expression'>

## Local Lax Friedrichs/Rusanov Riemann Solver Conservative

In [5]:
llf_dict = {s_l: -s, s_r: s}

In [14]:
rhs = (left_cell + right_cell).subs(llf_dict).simplify_full()
solve(rhs == 0, q_s)

[q_s == 1/2*((q_l + q_r)*s + f_l - f_r)/s]

In [25]:
rhs = (s_1 + s_2 + s_3 + s_4 + s_5 + s_6 + s_7 + s_8).subs(llf_dict)
solve(rhs == 0, s)

[s == -(f_l - f_r)/(q_l + q_r)]

In [6]:
q_s_dict = {q_s: 1/2*((q_l + q_r)*s + f_l - f_r)/s}
rhs = left_cell - right_cell
sol2 = solve(rhs == 0, f_s)
print(sol2[0].subs(q_s_dict).subs(llf_dict).simplify_full())

f_s == 1/2*(q_l - q_r)*s + 1/2*f_l + 1/2*f_r
